# Introduction

This notebook reproduces experiments reported in the paper "Leveraging Citizen Science and Low Cost Recorders for Acoustic Monitoring of Bird Species: A Case Study in Kenya" by Ciira wa Maina. The aim of the work is to investigate the use of acoustic models trained using data obtained from citizen scientists in automatic bird species recognition. We use data from the citizen science website [Xeno Canto](http://www.xeno-canto.org/) to train models for bird species recognition and test these models on data collected at the [Dedan Kimathi University of Technology](https://www.dkut.ac.ke/) [wildlife conservancy](https://conservancy.dkut.ac.ke/).

These audio data were collected using a low cost Raspberry Pi based recorder and are available on [Data Dryad]( http://dx.doi.org/10.5061/dryad.69g60). The recordings are described in the paper ["A Bioacoustic Record of a Conservancy in the Mount Kenya Ecosystem."](https://bdj.pensoft.net/articles.php?id=9906)

## Download Data

## Data processing
Let's get information on bird species present in the audio recordings. 

In [ ]:
#all necessary imports
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
import scipy as sp
from scipy import signal
import scipy.io.wavfile
import librosa
import librosa.display
import urllib.request
import pandas as pd
import os
import json
import requests
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import pairwise_distances,accuracy_score,precision_score,recall_score, f1_score,hamming_loss,classification_report
from sklearn.model_selection import KFold,train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve,auc
from tabulate import tabulate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score
from sklearn import svm
import oskmeans as skm
import csv
import random
import time

np.random.seed(123)#for reproducibility
t1=time.time()

#Download information on all species observed at DeKUWC (both point counts and acoustically). These has information on codes
allspecies_table_url='http://bdj.pensoft.net//lib/ajax_srv/article_elements_srv.php?action=download_table_csv&instance_id=3276052&article_id=9906'
allspecies_file_name='all_species.csv'
urllib.request.urlretrieve(allspecies_table_url,allspecies_file_name)

#Download list of species in foreground and background of recordings ( data in the paper)
foreground_table_url='http://bdj.pensoft.net//lib/ajax_srv/article_elements_srv.php?action=download_table_csv&instance_id=3276675&article_id=9906'
foreground_file_name='foreground.csv'
background_table_url='http://bdj.pensoft.net//lib/ajax_srv/article_elements_srv.php?action=download_table_csv&instance_id=3276676&article_id=9906'
background_file_name='background.csv'
urllib.request.urlretrieve(foreground_table_url,foreground_file_name);
urllib.request.urlretrieve(background_table_url,background_file_name);

Using the list of foreground and background recordings, we will get a list of the 36 species identified in the annotated recordings. We also change the entry for the Common Bulbul (*Pycnonotus barbatus*) to Dark-capped Bulbul (*Pycnonotus tricolor*) to correspond to the IOC taxonomy used in Xeno-canto.

In [ ]:
df_allspecies=pd.read_csv('all_species.csv',sep=';')
df_allspecies
df_foreground=pd.read_csv('foreground.csv',sep=';')
df_background=pd.read_csv('background.csv',sep=';')
species= set(df_foreground['Scientific Name'])#.union(set(df_background['Scientific Name']))
species=list(species)
species.sort()
print(len(species))
species[species.index('Pycnonotus barbatus')]='Pycnonotus tricolor'
df_allspecies.loc[df_allspecies['Scientific Name']=='Pycnonotus barbatus','Four-Letter Code']
df_allspecies.loc[df_allspecies['Scientific Name']=='Pycnonotus barbatus','Four-Letter Code']='DCBU'
df_allspecies.loc[df_allspecies['Scientific Name']=='Pycnonotus barbatus','Common Name']='Dark-capped Bulbul'
df_allspecies.loc[df_allspecies['Scientific Name']=='Pycnonotus barbatus','Scientific Name']='Pycnonotus tricolor'
all_species_sp=list(df_allspecies['Scientific Name'])
all_species_cn=list(df_allspecies['Common Name'])
all_species_code=list(df_allspecies['Four-Letter Code'])

For each of the species, we will examine xeno-canto and determine if there are more than 10 recordings deposited on the website which are jugded to be of quality C or better on an A-E scale. If this is the case, we will download the data and use it to train models. Each time this is run it may download recent uploads.

In [ ]:
data_dir='/home/ciira/Documents/Research/BiodiversityMonitoring/repo/acoustic-species-detection/data/mp3/'
#get recordings already present
XC_file=os.listdir(data_dir)
min_rec=10

#download
recording_data=[]
number_recordings=[]
for sp in species:
    webaddress='http://www.xeno-canto.org/api/2/recordings?query='+sp+' cnt:kenya'
    response=requests.get(webaddress)
    num_rec=0
    if response.status_code==200:
        data=response.json()
        rec=data['recordings']
        print ('Successful Query',all_species_cn[all_species_sp.index(sp)],int(data['numRecordings']))
        if len(rec)>0:
            for j in range(len(rec)):
                if rec[j]['rec']!='Ciira Maina' and rec[j]['q']!='no score':#Ignore recordings by author to avoid possible 'known individuals' and recordings with no score
                    #print(sp,rec[j]['id'],rec[j]['rec'],rec[j]['url'])
                    num_rec+=1
        number_recordings.append(num_rec)                    
        if num_rec>=min_rec:
            for j in range(len(rec)):
                if rec[j]['rec']!='Ciira Maina' and rec[j]['q']!='no score':#Ignore recordings by author to avoid possible 'known individuals' and recordings with no score
                    #print(sp,rec[j]['id'],rec[j]['rec'],rec[j]['url'])
                    if XC_file.count(str(rec[j]['id'])+'.mp3')==0:#recording is not present
                        print ('Downloading file ',rec[j]['id'],'for',sp)
                        filename=data_dir+rec[j]['id']+'.mp3'
                        urllib.request.urlretrieve(rec[j]['file'],filename)
                    recording_data.append([rec[j]['id'],rec[j]['en'],sp,rec[j]['rec'],rec[j]['url']])

df_xc=pd.DataFrame.from_records(recording_data,columns=['Xeno-canto ID','Common Name','Scientific Name','Recordist','url'])
recog_species=list(set(df_xc['Scientific Name']))
recog_species.sort()
recog_species_cn=[all_species_cn[all_species_sp.index(recog_species[i])] for i in range(len(recog_species))]
recog_species_code=[all_species_code[all_species_sp.index(recog_species[i])] for i in range(len(recog_species))]
recog_species_numrec=[number_recordings[species.index(recog_species[i])] for i in range(len(recog_species))]

#save
rows = zip(recog_species,recog_species_cn,recog_species_code,recog_species_numrec)
with open("recog_species.csv", "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)


df_xc.to_csv('recordings.csv',index=False)

We convert the mp3 files to wav.

In [ ]:
mp3_dir = 'data/mp3/'
wav_dir = 'data/wav-16kHz/'
mp3s=os.listdir(mp3_dir)
wavs=os.listdir('data/wav-16kHz/')

for filename in mp3s:
    if filename.split('.')[0]+'.wav' not in wavs:
        print('Converting...',filename)
        cmd ='sox '+mp3_dir+filename+' -r 16000 -c 1 '+wav_dir+filename.split('.')[0]+'.wav'
        os.system(cmd)



## Preliminary visualisations
Now we can examine some of the audio data and plot some spectrograms. Figure 3a in the paper.

In [ ]:
#audio parameters
Nfft=512
Noverlap=256
f_min=300
f_max=8000
num_mel=80
summary='mean_std'
learned_features=False
patch_width=8
patch_height=num_mel
#indx=np.random.randint(0,len(df_xc)-1)df_xc['Common Name']
indx=244
print('The recording is XC%d from Xeno-canto.'%int(df_xc['Xeno-canto ID'][indx]))

x = scipy.io.wavfile.read(wav_dir+df_xc['Xeno-canto ID'][indx]+'.wav')
fs = float(x[0])
x2 = x[1] / 2.**15
x2 = x2 / np.max(np.abs(x2))
Spec = librosa.stft(x2,n_fft=Nfft, hop_length=Noverlap)
librosa.display.specshow(librosa.amplitude_to_db(Spec, ref=np.max),  cmap='gray_r',y_axis='linear', x_axis='time',sr=fs,hop_length=Noverlap);
plt.savefig('../../figures/XC_HT_spec.jpg',dpi=300)

The corresponding melspectrogram. Figure 3b in the paper

In [ ]:
from matplotlib import gridspec
S = librosa.feature.melspectrogram(S=np.abs(Spec)**2, sr=fs, n_mels=num_mel,fmin=f_min,fmax=f_max)
librosa.display.specshow(librosa.power_to_db(S, ref=np.max), cmap='gray_r',y_axis='mel', x_axis='time',sr=fs,hop_length=Noverlap,fmin=f_min,fmax=f_max)

gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 
ax0 = plt.subplot(gs[0])
librosa.display.specshow(librosa.power_to_db(S, ref=np.max), cmap='gray_r',y_axis='mel', x_axis='time',sr=fs,hop_length=Noverlap,fmin=f_min,fmax=f_max)
ax1 = plt.subplot(gs[1])
ax1.plot(np.mean(S,1),np.arange(num_mel))
ax1.set_yticklabels('');
plt.savefig('../../figures/XC_HT_mel.jpg',dpi=300)

We now obtain features from the data for use in training and testing our classifiers. We will use the following features:
1. The melspectrogram summarised using the mean and standard deviation
2. Learned features from time-frequency patches of the melspectrogram

## Melspectrogram Features

In [ ]:
def avg_melspec(filename,Nfft,Noverlap,f_min,f_max,num_mel,summary):
    x = scipy.io.wavfile.read(filename)
    x2 = x[1] / 2.**15
    x2 = x2 / np.max(np.abs(x2))
    fs = float(x[0])
    Spec = librosa.stft(x2,n_fft=Nfft, hop_length=Noverlap)
    Spec=np.abs(Spec)
    S = librosa.feature.melspectrogram(S=np.abs(Spec)**2, sr=fs, n_mels=num_mel,fmin=f_min,fmax=f_max)
    S/=S.max()
    meanS=np.mean(S,1)
    
    if summary=='mean':
        return meanS
    elif summary=='mean_std':
        return np.concatenate((meanS,np.std(S,1)))
    elif summary=='max':
        return np.max(S,1)
        


def melspec_time_frequency_patch(filename,Nfft,Noverlap,f_min,f_max,num_mel,patch_height,patch_width):
    x = scipy.io.wavfile.read(filename)
    x2 = x[1] / 2.**15
    x2 = x2 / np.max(np.abs(x2))
    fs = float(x[0])
    Spec = librosa.stft(x2,n_fft=Nfft, hop_length=Noverlap)
    S = librosa.feature.melspectrogram(S=np.abs(Spec)**2, sr=fs, n_mels=num_mel,fmin=f_min,fmax=f_max)
    S/=S.max()
    feature=[]

    for i in range(0,S.shape[0]-patch_height+1,patch_height):
        for j in range(0,S.shape[1]-patch_width+1,patch_width):
            TF_patch=S[i:i+patch_height,j:j+patch_width]
            feature.append(np.ndarray.flatten(TF_patch))
        
    return np.array(feature)


features=[]
labels=[]


for i in range(len(df_xc)):
    print('Processing XC%d'%int(df_xc['Xeno-canto ID'][i]))
    features.append(avg_melspec(wav_dir+df_xc['Xeno-canto ID'][i]+'.wav',Nfft,Noverlap,f_min,f_max,num_mel,summary))
    labels.append(recog_species.index(df_xc['Scientific Name'][i]))


X=np.array(features)
labels=np.array(labels)
y=label_binarize(labels,np.unique(labels))

We now test the performance of a random forest classifier by determining the precision, recall and F1 score for each species using 20 random splits of the data into 70% for training and 30% for testing.



In [ ]:
NT=500 #number of trees
num_trial=20
test_proportion=.3
species_precision=np.zeros((len(recog_species),num_trial))
species_recall=np.zeros((len(recog_species),num_trial))
species_f1=np.zeros((len(recog_species),num_trial))
trial_pred_scores=[]
trial_y_test=[]
for trial in range(num_trial):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=test_proportion)
    print('Training random forest classifier...')
    clf= OneVsRestClassifier(RandomForestClassifier(n_estimators=NT,criterion='entropy'))
    clf.fit(X_train,y_train)
    result=clf.predict(X_test)
    scores=clf.predict_proba(X_test)
    top_res=np.zeros(result.shape)
    top_3_res=np.zeros(result.shape)
    trial_pred_scores.append(scores)
    trial_y_test.append(y_test)

    print('Screening...')
    for i in range(result.shape[0]):
        line=''
        for j in range(len(y_test[i,:].nonzero()[0])):
            line+=recog_species_code[y_test[i,:].nonzero()[0][j]]+' '
        line+=';'

        indx=np.argsort(scores[i,:])[::-1][0:3].tolist()
        top_res[i,indx[0]]=1
        top_3_res[i,indx]=1
        file_species=[recog_species_code[i] for i in indx]
        for j in range(len(file_species)):
            line+=file_species[j]+' '

        print(line)
    for i in range(len(recog_species)):
        species_precision[i,trial]=precision_score(y_test[:,i],top_res[:,i])
        species_recall[i,trial]=recall_score(y_test[:,i],top_res[:,i])
        species_f1[i,trial]=f1_score(y_test[:,i],top_res[:,i])
         
print(classification_report(y_test, top_res, target_names=recog_species_cn))


Produce box plots of the precision, recall and F1 score. Figure 4 in the paper.

In [ ]:
plt.figure()
plt.boxplot(species_precision.T)
plt.xticks(range(1,len(recog_species)+1), recog_species_code, rotation='vertical')
plt.ylabel('Precision')
plt.savefig('../../figures/xc_precision_mel.jpg',dpi=300)

plt.figure()
plt.boxplot(species_recall.T)
plt.xticks(range(1,len(recog_species)+1), recog_species_code, rotation='vertical')
plt.ylabel('Recall')
plt.savefig('../../figures/xc_recall_mel.jpg',dpi=300)

plt.figure()
plt.boxplot(species_f1.T)
plt.xticks(range(1,len(recog_species)+1), recog_species_code, rotation='vertical')
plt.ylabel('F1 Score')
plt.savefig('../../figures/xc_f1_mel.jpg',dpi=300)




Create a table with the median precision, recall and F1 score. Table 2 in the paper.

In [ ]:
df_species_metrics=pd.DataFrame(list(zip(recog_species_code,np.median(species_precision,1).tolist(),np.median(species_recall,1).tolist(), np.median(species_f1,1).tolist())),columns=['Species','Precision','Recall','F1 Score'])
df_species_metrics=df_species_metrics.sort_values(['Precision'], ascending=False)
df_species_metrics=df_species_metrics.reset_index(drop=True)
print(df_species_metrics.round(2))

Screen DeKUWC recordings.


In [ ]:
#Train with all the xeno-canto data
clf_mel= OneVsRestClassifier(RandomForestClassifier(n_estimators=NT,criterion='entropy'))
clf_mel.fit(X,y)

df_dekuwc=pd.read_csv('data/DeKUWC/PiRecordingsAnnotation.csv')
dekut_wavs=os.listdir('data/DeKUWC/wav')
dekut_wavs.sort()
topN=3
dekuwc_true_label=np.zeros((len(dekut_wavs),len(recog_species_code)))

dekuwc_pred_scores=[]
annotation=[]
recog_result=[]
for filename in dekut_wavs:
    print('Processing',filename)
    df=df_dekuwc.loc[df_dekuwc['Filename']==filename]
    
    features=avg_melspec('data/DeKUWC/wav/'+filename,Nfft,Noverlap,f_min,f_max,num_mel,summary)
    
    file_result=clf_mel.predict_proba(np.array(features).reshape(1, -1))
    dekuwc_pred_scores.append(file_result[0,:])
    indx=np.argsort(file_result)[0][::-1][0:topN].tolist()
    recog_file_species=[recog_species_code[i] for i in indx]
    true_file_species=df.iloc[0,1].split(';')
    if 'COBU' in true_file_species:
        true_file_species[true_file_species.index('COBU')]='DCBU'
    for i in range(len(true_file_species)):
        if true_file_species[i] in recog_species_code:
            dekuwc_true_label[dekut_wavs.index(filename),recog_species_code.index(true_file_species[i])]=1
    
    dd=recog_file_species[0]
    for i in range(1,len(recog_file_species)):
        dd+=';'
        dd+=recog_file_species[i]
    annotation.append(df.iloc[0,1])
    recog_result.append(dd)
    #print(true_file_species+[';']+recog_file_species)
    
np.array(dekuwc_pred_scores)   
df_screening = pd.DataFrame(list(zip(dekut_wavs, annotation, recog_result)),columns=['Filenames','True Species','Recognition Result'])
df_screening.to_csv('screening_result_mel.csv',index=False)


Show the screening result for a selection of five files. Table 4 of the paper (melspecrogram feature result)

In [ ]:
print(df_screening.loc[df_screening['Filenames'].isin(['1-2016-01-05-10-40-01.wav','1-2016-01-05-11-10-01.wav','2-2016-01-06-07-45-01.wav','4-2016-01-06-11-45-01.wav','6-2016-01-29-12-35-01.wav'])].reset_index(drop=True))


Show the classification report. Table 3 of the paper. This is obtained by selecting a threshold that optimizes the F1 score for each species and reports that species as present when its probability exceeds this threshold.

In [ ]:
species_threshold=np.zeros(len(recog_species_code))
species_max_f1=np.zeros(len(recog_species_code))
species_max_precision=np.zeros(len(recog_species_code))
species_max_recall=np.zeros(len(recog_species_code))
thresh=np.linspace(0,1,100)
dekuwc_pred_scores=np.vstack(dekuwc_pred_scores)
for sp_code in recog_species_code:
    f1_scores=np.zeros(len(thresh))
    for i in range(len(thresh)):
        f1_scores[i]=f1_score(dekuwc_true_label[:,recog_species_code.index(sp_code)],dekuwc_pred_scores[:,recog_species_code.index(sp_code)]>thresh[i]);
    
    species_max_f1[recog_species_code.index(sp_code)]=np.max(f1_scores)
    species_threshold[recog_species_code.index(sp_code)]=thresh[np.argmax(f1_scores)]
    species_max_precision[recog_species_code.index(sp_code)]=precision_score(dekuwc_true_label[:,recog_species_code.index(sp_code)],dekuwc_pred_scores[:,recog_species_code.index(sp_code)]>thresh[np.argmax(f1_scores)])
    species_max_recall[recog_species_code.index(sp_code)]=recall_score(dekuwc_true_label[:,recog_species_code.index(sp_code)],dekuwc_pred_scores[:,recog_species_code.index(sp_code)]>thresh[np.argmax(f1_scores)])

df_dekuwc_metrics=pd.DataFrame(list(zip(recog_species_code,species_max_precision.tolist(),species_max_recall.tolist(), species_max_f1.tolist())),columns=['Species','Precision','Recall','F1 Score'])
df_dekuwc_metrics=df_dekuwc_metrics.sort_values(['F1 Score'], ascending=False)
df_dekuwc_metrics=df_dekuwc_metrics.reset_index(drop=True)
print(df_dekuwc_metrics.round(2))

In [ ]:
print('Time taken for melspectrogram features is %.2f minutes'%((time.time()-t1)/60))

## Hartlaub's Turaco Screening

We now screen all the 2701 recordings obtained from the DeKUWC for presence of the Harltaub's Turaco. 


In [ ]:
t1=time.time()
df_all_dekuwc=pd.read_csv('data/DeKUWC/AllLocations_Annotation.csv')
all_filenames=list(df_all_dekuwc['Filename'])
all_dekuwc_pred_score=[]
for filename in all_filenames:
    print('Processing',filename)
    features=avg_melspec('../../data/DeKUWC_PiRecordings2016/wav/'+filename+'.wav',Nfft,Noverlap,f_min,f_max,num_mel,summary)
    file_result=clf_mel.predict_proba(np.array(features).reshape(1, -1))
    all_dekuwc_pred_score.append(file_result[0,:])
    
print('Time taken to screen all files is %.2f minutes'%((time.time()-t1)/60))

Obtain the Hartlaub's Turaco recordings and plot spectrograms

In [ ]:
all_dekuwc_pred_score=np.vstack(all_dekuwc_pred_score)
all_dekuwc_pred_label=np.array(all_dekuwc_pred_score>species_threshold).astype(int)
hatu_file=[]
plot_fig=True
for filename in all_filenames:
    if all_dekuwc_pred_label[all_filenames.index(filename),recog_species_code.index('HATU')]:
        print(filename)
        hatu_file.append(filename)
        if plot_fig:
            x = scipy.io.wavfile.read('../../data/DeKUWC_PiRecordings2016/wav/'+filename+'.wav')
            fs = float(x[0])
            x2 = x[1] / 2.**15
            x2 = x2 / np.max(np.abs(x2))
            Spec = librosa.stft(x2,n_fft=Nfft, hop_length=Noverlap)
            librosa.display.specshow(librosa.amplitude_to_db(Spec, ref=np.max),  cmap='gray_r',y_axis='linear', x_axis='time',sr=fs,hop_length=Noverlap);
            plt.savefig('HATU-Spectrograms/'+filename+'.jpg',dpi=300)
            plt.clf()
            
df_screening = pd.DataFrame(list(zip(hatu_file)),columns=['Filenames'])
df_screening.to_csv('HATU_screening_result_mel.csv',index=False)